# Densenet121 Pre trained to test

This jupyter notebook has the objective to, not only retrieve the accuracies of the Densenet121 pretrained, but to obtain also <br>
the layer features before the last classification layer.

In [1]:
 #Import necessary modules
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import transforms
plt.rcParams['figure.figsize'] = [20, 12]

### Set the path to here

Make sure the setup the paths properly!

In [2]:
#Path to assign tests (copy path directly)
notebooks_path = os.getcwd() # OR MAYBE has to be set manually depending your computer

#Set the path to this working directory
os.chdir(notebooks_path)
print(os.getcwd())

import sys
#Append the path the src folder
sys.path.append(os.path.join(os.getcwd(), os.pardir, "src"))

c:\Users\juan.pablo\Documents\temporal_school_rel\CS231N-Final-Proj\notebooks


### Import the necessary module for downloading

Note for this: EVERYTIME There is a change inside the download <br>
the changes inside the file would only be shown if the jupyter kernel is restarted. <br>


In [3]:
# Import the necessary modules
from utils import CXReader, DfReader

### Set the data path

In [4]:
# Create the data path
df_path = os.path.join(notebooks_path, os.pardir, "data")
data_path = os.path.join(df_path, "images")

### Get the dataframes of the data
First, lets obtain the dataframes for the data and check that all metadata <br>
information has been set up properly. <br>

In [5]:
#Create a dataframe compiler
df_compiler = DfReader()

#set the path and retrieve the dataframes
df_compiler.set_folder_path(df_path)

#Get the dataframe holder and names
dfs_holder, dfs_names = df_compiler.get_dfs()

  0%|          | 0/4 [00:00<?, ?it/s]

The file: miccai2023_nih-cxr-lt_labels_test.csv has been retrieved


100%|██████████| 4/4 [00:00<00:00,  8.59it/s]

The file: miccai2023_nih-cxr-lt_labels_train.csv has been retrieved
The file: miccai2023_nih-cxr-lt_labels_val.csv has been retrieved


# Read the images and labels

Also, obtain DataLoaders for test, train, and validation datasets using <br>
the Dataloader class from pytorch.

In [6]:
print(torch.cuda.is_available())

False


In [7]:
# Get the device if cuda or not
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Define a transformations for the VGGnet16 (requires a 224,224)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256
    transforms.CenterCrop((224, 224)),  # Center crop to 224x224
    transforms.ToTensor(),
])

#Create datasets and dataloaders
test_dataset = CXReader(data_path=data_path, dataframe=dfs_holder[0], transform=transform, device=device)
train_dataset = CXReader(data_path=data_path, dataframe=dfs_holder[1], transform=transform,device=device)
val_dataset = CXReader(data_path=data_path, dataframe=dfs_holder[2], transform=transform, device=device)

#Sampled images from train to see single shape
samp3_image, label3 = train_dataset[1]
print("Shape of a single image and its labels")
print(f"Image: {samp3_image.shape}, labels: {label3.shape}")

#With batch size of 32, and shuffle true, and num workers = 4
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,  num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=2)

#Iterate inside a batch
for idx, batch in enumerate(train_loader):
    print(f"batch number: {idx}")
    images, labels = batch
    print("Shape of batch of images and labels")
    print(f"Images: {images.shape}, labels: {labels.shape}")
    if idx == 5:
        print("It can iterate through all batches")
        break

Shape of a single image and its labels
Image: torch.Size([3, 224, 224]), labels: torch.Size([20])
batch number: 0
Shape of batch of images and labels
Images: torch.Size([32, 3, 224, 224]), labels: torch.Size([32, 20])
batch number: 1
Shape of batch of images and labels
Images: torch.Size([32, 3, 224, 224]), labels: torch.Size([32, 20])
batch number: 2
Shape of batch of images and labels
Images: torch.Size([32, 3, 224, 224]), labels: torch.Size([32, 20])
batch number: 3
Shape of batch of images and labels
Images: torch.Size([32, 3, 224, 224]), labels: torch.Size([32, 20])
batch number: 4
Shape of batch of images and labels
Images: torch.Size([32, 3, 224, 224]), labels: torch.Size([32, 20])
batch number: 5
Shape of batch of images and labels
Images: torch.Size([32, 3, 224, 224]), labels: torch.Size([32, 20])
It can iterate through all batches


### Load the densenet121 pretrained model

Check if you have GPU Envidia! Else, use the cpu

In [10]:
#Load the pretrained model
densenet121 = models.densenet121(pretrained = True)

c:\Users\juan.pablo\Documents\temporal_school_rel\CS231N-Final-Proj\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\juan.pablo\Documents\temporal_school_rel\CS231N-Final-Proj\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### See the densenet121.features architecture and get the parameter shapes

In [11]:
print(densenet121.features)
print([x.shape for x in densenet121.features.parameters()])

Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 128, ke

### See the densenet121 classifier parameters and weights

In [12]:
print(densenet121.classifier)
print([x.shape for x in densenet121.classifier.parameters()])

Linear(in_features=1024, out_features=1000, bias=True)
[torch.Size([1000, 1024]), torch.Size([1000])]


In [13]:
# NEW CODE CELL to conduct fine-tuning on Vggnet16 only on the last (Linear) layer

# First, freeze all the parameters
for param in densenet121.parameters():
    param.requires_grad = False

In [14]:
from copy import copy

# Modify the last layer for the last 20 classes
num_classes = 20  # Number of classes for your specific task
num_features = copy(densenet121.classifier.in_features) #Get all of the features after convolutional layers

print(num_classes)
print(num_features)

20
1024


In [15]:
print(num_features)

#Obtain the same classifier you got befor with lower number of classes, so we can pretrain it
densenet121.classifier = nn.Sequential(
    nn.Linear(num_features, 4096, bias=True),
    nn.ReLU(inplace= True),
    nn.Dropout(0.5, inplace=False),
    nn.Linear(4096, 4096, bias=True),
    nn.ReLU(inplace= True),
    nn.Dropout(0.5, inplace=False),
    nn.Linear(4096, num_classes, bias=True),
    nn.ELU(inplace=True)
)

print(densenet121.classifier)

1024
Sequential(
  (0): Linear(in_features=1024, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=20, bias=True)
)


In [17]:
# NEW CODE CELL

# Create state_dict path
model_dict_path = os.path.join(notebooks_path, os.pardir, "models")

if os.path.exists(model_dict_path) == False:
    os.mkdir(model_dict_path)

In [16]:
# NEW CODE CELL to perform fine-tuning
#print([x.shape for x in resnet18.classifier[-6].parameters()])

import torch.optim as optim
resnet18 = densenet121.to(device)
resnet18.train()
params_to_update = [resnet18.classifier.parameters()]
print(params_to_update)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(*params_to_update, lr=0.01)

def finetune_model(model, data_loader, num_epochs, device:str):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-------------')
            
        for idx, batch in enumerate(data_loader):
            images_inputs, images_labels = batch
            images_inputs, images_labels = images_inputs.to(device), images_labels.to(device)

            # Convert labels to float type (also need to move to CUDA again!)
            images_labels = images_labels.to(torch.float64)

            # initialize optimizer
            optimizer.zero_grad()            
            outputs = model(images_inputs)
            
            # compute loss
            loss = criterion(outputs, images_labels)
            
            # predict labels
            pred_labels = (outputs > 0.5).float()

            # Calculate TP, FP, TN, FN and accuracy
            TP = torch.sum((pred_labels == 1) & (images_labels == 1)).item()
            FP = torch.sum((pred_labels == 1) & (images_labels == 0)).item()
            TN = torch.sum((pred_labels == 0) & (images_labels == 0)).item()
            FN = torch.sum((pred_labels == 0) & (images_labels == 1)).item()
            accuracy = ((TP + TN) / (TP + FP + TN + FN)) * 100.0                

            loss.backward()
            print(f"iter {idx} ---  Loss: {loss}    Accuracy: {accuracy}")
            optimizer.step()
        
        # Save parameters for each epoch
        torch.save(model.state_dict(), os.path.join(model_dict_path, "densenet121_finetune_params.pth"))

[<generator object Module.parameters at 0x0000020670D7DEE0>]


In [ ]:
# Let's do fine-tuning
finetune_model(model=resnet18, data_loader=train_loader, num_epochs=5, device=device)

Epoch 1/5
-------------
iter 0 ---  Loss: 3.6257742643356323    Accuracy: 93.28125
iter 1 ---  Loss: 3.0337693095207214    Accuracy: 94.84375
iter 2 ---  Loss: 1.7913618851453066    Accuracy: 96.71875
iter 3 ---  Loss: 2.435753282159567    Accuracy: 92.34375
iter 4 ---  Loss: 2.495018010959029    Accuracy: 90.3125
iter 5 ---  Loss: 1.6206142427399755    Accuracy: 89.84375
iter 6 ---  Loss: 3.195416498463601    Accuracy: 88.4375
iter 7 ---  Loss: 3.3637816635891795    Accuracy: 86.25
iter 8 ---  Loss: 2.897922497242689    Accuracy: 83.75
iter 9 ---  Loss: 2.1838474180549383    Accuracy: 83.90625
iter 10 ---  Loss: 2.2920630695298314    Accuracy: 85.46875
iter 11 ---  Loss: 2.9979859087616205    Accuracy: 83.125
iter 12 ---  Loss: 2.0618113297969103    Accuracy: 84.6875
iter 13 ---  Loss: 2.021247980184853    Accuracy: 84.53125
iter 14 ---  Loss: 2.60822572093457    Accuracy: 84.6875
iter 15 ---  Loss: 2.2241524569690228    Accuracy: 83.90625
iter 16 ---  Loss: 2.550775147974491    Accur

KeyboardInterrupt: 

In [ ]:
densenet121.load_state_dict(torch.load(os.path.join(model_dict_path, "vgg16_finetune_params.pth")))

<All keys matched successfully>

### Create a function that would evaluate the model.

Make sure it outputs all of the accuracies of all 20 conditions. <br>

In [ ]:
import torch.nn.functional as F

def evaluate_model(model, data_loader, limit:int, device:str):
    """
    Instance method that would evaluate with a given
    data loader, the accuracies obtained by the VGGNET16
    """
    model.eval()
    threshold = 0.5
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    #Use no grad to not perform backpropagation for inference time
    with torch.no_grad():
        #Iterate through each of the images and labels
        
        # Calculate the total numbers for metrics
        TP, FP, TN, FN = 0.0, 0.0, 0.0, 0.0
        for idx, batch in enumerate(data_loader):
    
            #See if it works
            images_inputs, images_labels = batch
            images_inputs, images_labels = images_inputs.to(device), images_labels.to(device)

            #Print the shape of each one of them
            print(f"Inputs shape: {images_inputs.shape}, Labels shape: {labels.shape}")

            #Send the outputs to model in device
            outputs = model(images_inputs)

            #Binarize the output with threshold
            pred_labels = (outputs > threshold).float()

            # Calculate batch-wise TP, FP, TN, FN
            b_TP = torch.sum((pred_labels == 1) & (images_labels == 1)).item()
            b_FP = torch.sum((pred_labels == 1) & (images_labels == 0)).item()
            b_TN = torch.sum((pred_labels == 0) & (images_labels == 0)).item()
            b_FN = torch.sum((pred_labels == 0) & (images_labels == 1)).item()
            TP += b_TP
            FP += b_FP
            TN += b_TN
            FN += b_FN

        #_, predicted = torch.max(outputs, 1)  # Get the index of the maximum log-probability
        accuracy = ((TP + TN) / (TP + FP + TN + FN)) * 100.0
        precision = (TP / (TP + FP)) * 100.0 if (TP + FP) > 0 else 0.0
        recall = (TP / (TP + FN)) * 100.0 if (TP + FN) > 0 else 0.0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        print("Accuracy: {:.2f}%".format(accuracy))
        print("Precision: {:.2f}%".format(precision))
        print("Recall: {:.2f}%".format(recall))
        print("F1 Score: {:.2f}%".format(f1_score))

            # accuracies.append(accuracy)
            # precisions.append(precision)
            # recalls.append(recall)
            # f1_scores.append(f1_score)

            # if idx == limit:
            #     print("Limit reached")
            #     break
    return accuracies, precisions, recalls, f1_scores

In [ ]:
# Evaluate on the eval set
accuracies, precisions, recalls, f1_scores = evaluate_model(resnet18, test_loader, 5, device=device)

Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32, 20])
Inputs shape: torch.Size([32, 3, 224, 22